- code based on : https://yeomko.tistory.com/8
- study based on : https://yeomko.tistory.com/5

In [1]:
import pandas as pd

In [40]:
movies = pd.read_csv('C:/Users/Sooyeon/Desktop/data_science/추천시스템/week1&2/movies.csv')
ratings = pd.read_csv('C:/Users/Sooyeon/Desktop/data_science/추천시스템/week1&2/ratings.csv')
tags = pd.read_csv('C:/Users/Sooyeon/Desktop/data_science/추천시스템/week1&2/tags.csv')

In [3]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [41]:
# movielense 사용할 user*item matrix 만들기
ratings.drop('timestamp', axis=True, inplace=True)
r_m = ratings.merge(movies, on='movieId')
matrix_= pd.pivot_table(r_m, index=['userId'],columns=['title'],values='rating').fillna(0)
matrix_.head(5)

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
print(matrix_.columns)
print(matrix_.index)

Index(['"Great Performances" Cats (1998)', '$9.99 (2008)',
       ''Hellboy': The Seeds of Creation (2004)',
       ''Neath the Arizona Skies (1934)', ''Round Midnight (1986)',
       ''Salem's Lot (2004)', ''Til There Was You (1997)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '(500) Days of Summer (2009)',
       ...
       'Zulu (1964)', 'Zulu (2013)', '[REC] (2007)', 'eXistenZ (1999)',
       'loudQUIETloud: A Film About the Pixies (2006)', 'xXx (2002)',
       'xXx: State of the Union (2005)', '¡Three Amigos! (1986)',
       'À nous la liberté (Freedom for Us) (1931)', 'İtirazım Var (2014)'],
      dtype='object', name='title', length=9064)
Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            662, 663, 664, 665, 666, 667, 668, 669, 670, 671],
           dtype='int64', name='userId', length=671)


In [42]:
matrix_= np.copy(matrix_)
matrix_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 학습 파라미터 초기화
- r_lambda : normalization 정규화 파라미터
- alpha: confidence level
- nf: latent vector의 dimention

In [43]:
r_lambda = 40
nf = 200
alpha = 40

- nu: number of user
- ni: number of items
- nf: dimension of latent vector

In [44]:
matrix_.shape

(671, 9064)

In [45]:
nu = matrix_.shape[0]
ni = matrix_.shape[1]

ALS는 먼저 사용자 혹은 아이템의 latent factor 행렬을 아주 작은 랜던값으로 초기화한다. 그다음 둘중 하나를 상수처럼 고정시켜 loss function을 convex function으로 만들다

In [8]:
import numpy as np

In [9]:
x = np.random.rand(nu, nf)* 0.01
y = np.random.rand(ni, nf)* 0.01

In [46]:
x = x.astype('float')
print(x)

[[0.00433136 0.00999828 0.00275712 ... 0.00068883 0.00085265 0.00721753]
 [0.00781024 0.00754897 0.00417296 ... 0.00476689 0.00458411 0.00514257]
 [0.00706968 0.00279575 0.00115091 ... 0.00282982 0.00590517 0.00379658]
 ...
 [0.00995546 0.00473598 0.00768758 ... 0.00620837 0.003674   0.0076509 ]
 [0.00419394 0.00491855 0.00572116 ... 0.00601563 0.00168116 0.00659108]
 [0.00241249 0.00967447 0.00810298 ... 0.00877086 0.00198811 0.0002199 ]]


In [47]:
y = y.astype('float')
print(y)

[[0.00056807 0.00738724 0.00205871 ... 0.00802337 0.00914899 0.00020781]
 [0.00945457 0.00592572 0.00107739 ... 0.00881362 0.00621612 0.00897002]
 [0.00574573 0.00727597 0.00065808 ... 0.00058213 0.00013154 0.00892277]
 ...
 [0.00428934 0.00869715 0.00322333 ... 0.00118925 0.00582626 0.00568001]
 [0.00496684 0.00363821 0.0072954  ... 0.00090918 0.00927398 0.00495877]
 [0.00570706 0.00358155 0.00917356 ... 0.00391759 0.00317079 0.00497568]]


선호도 행렬 p 설정 (convert orginal rating matrix R into P)
- Pui = 1 if Rui>0
- Pui = 0 if Rui=0

In [48]:
P = np.copy(matrix_)
P[P>0] = 1
print(P)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
P.shape

(671, 9064)

신뢰도 행렬 c 설절
- 학습용 평점 테이블에 confidence level을 적용한 c행렬을 구한다
- Cui = 1 + alpha * Rui

In [49]:
C = 1 + alpha * matrix_
print(C)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


loss function 설정
- 예측이 얼마나 정확한지를 측정할 loss function을 작성한다
- loss function의

In [50]:
def loss_function(C, P, xTy, x,y, r_rambda):
    predict_error = np.square(P - xTy)
    confidence_error = np.sum(C * predict_error)
    
    regularization = r_rambda * (np.sum(np.square(x) + np.sum(np.square(y))))
    
    total_loss = confidence_error + regularization
    
    return np.sum(predict_error), confidence_error, regularization, total_loss

optimizer 설정
- loss function을 최적화할 optimize function을 작성
- user latent factor optimizer
- item latent factor optimizer

In [58]:
def optimize_user(x, y, C, P, nu, nf, r_lambda):
    yT = np.transpose(y)
    
    for u in range(nu):
        Cu = np.diag(C[u])
        yT_Cu_y = np.matmul(np.matmul(yT, Cu), y)
        iI = np.dot(r_lambda, np.identity(nf))
        
        yT_Cu_pu = np.matmul(np.matmul(yT, Cu), P[u])
        x[u] = np.linalg.solve(yT_Cu_y + iI, yT_Cu_pu)
        
        
def optimize_item(x, y, C, P, ni, nf, r_lambda):
    xT = np.transpose(x)
    
    for i in range(ni):
        Ci = np.diag(C[:,i])
        xT_Ci_x = np.matmul(np.matmul(xT, Ci), x)
        iI = np.dot(r_lambda, np.identity(nf))
        
        xT_Ci_pi = np.matmul(np.matmul(xT, Ci), P[:, i])
        y[i] = np.linalg.solve(xT_Ci_x + iI, xT_Ci_pi)

train 
- usually ALS algorithm repeat train steps for 10 to 15 times

In [ ]:
predict_errors = []
confidence_errors = []
regularization_list = []
total_losses = []

for i in range(15):
    if i!=0:   
        optimize_user(x, y, C, P, nu, nf, r_lambda)
        optimize_item(x, y, C, P, ni, nf, r_lambda)
    predict = np.matmul(x, np.transpose(y))
    predict_error, confidence_error, regularization, total_loss = loss_function(C, P, predict, x,y, r_lambda)
   
    predict_errors.append(predict_error)
    confidence_errors.append(float(confidence_error))
    regularization_list.append(regularization)
    total_losses.append(total_loss)
    
    print('----------------step %d----------------' % i)
    print("predict error: %f" % predict_error)
    print("confidence error: %f" % confidence_errors[i])
    print("regularization: %f" % regularization)
    print("total loss: %f" % total_loss)
    
predict = np.matmul(x, np.transpose(y))
print('final predict')
print([predict])

----------------step 0----------------
predict error: 99160.208489
confidence error: 14133432.331339
regularization: 324373346.927312
total loss: 338506779.258651
----------------step 1----------------
predict error: 520174.121072
confidence error: 812055.420155
regularization: 52458435227.128365
total loss: 52459247282.548523


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.subplots_adjust(wspace=100.0, hspace=20.0)
fig = plt.figure()
fig.set_figheight(10)
fig.set_figwidth(10)
predict_error_line = fig.add_subplot(2, 2, 1)
confidence_error_line = fig.add_subplot(2, 2, 2)
regularization_error_line = fig.add_subplot(2, 2, 3)
total_loss_line = fig.add_subplot(2, 2, 4)

predict_error_line.set_title("Predict Error") 
predict_error_line.plot(predict_errors)

confidence_error_line.set_title("Confidence Error")
confidence_error_line.plot(confidence_errors)

regularization_error_line.set_title("Regularization")
regularization_error_line.plot(regularization_list)

total_loss_line.set_title("Total Loss")
total_loss_line.plot(total_losses)
plt.show()

돌아가는데 몇시간은 걸리겠는데????